# Idealized data

Here, we demonstrate the capabilities of {mod}`tams.idealized` for creating synthetic datasets for testing and experimentation.

In [ ]:
import warnings

import cartopy.io
import geopandas as gpd
import matplotlib.pyplot as plt

import tams
from tams.idealized import Blob, Field, Sim

warnings.filterwarnings("ignore", category=cartopy.io.DownloadWarning)

## Blob

A {class}`~tams.idealized.Blob` is an ellipse with parameters tendencies (set to zero by default) and a parabolic well.

The default {class}`~tams.idealized.Blob` is a circle.

In [ ]:
Blob().polygon

In [ ]:
Blob().ring

In [ ]:
Blob().to_geopandas().plot();

In [ ]:
_, ax = plt.subplots(layout="constrained", figsize=(4, 4))

ax.add_patch(Blob().to_patch(fc="forestgreen"))
ax.set_aspect("equal")
ax.autoscale()

The default {class}`~tams.idealized.Blob` has zero tendency in its parameters,
meaning that when {meth}`~tams.idealized.Blob.evolve` is called, it won't move or grow.

In [ ]:
Blob().get_tendency()

### Split

In [ ]:
Blob(center=(1, 2)).split()

In [ ]:
Field(Blob(center=(1, 2), width=6, height=2, angle=20).split(3)).to_geopandas().plot(fc="none");

### Merge

In [ ]:
b1 = Blob(width=6, height=2, angle=30)
b2 = Blob(width=8, height=4)
b3 = b1.merge(b2)

Field([b1, b2, b3]).to_geopandas().plot(fc="none");

## Field

A {class}`~tams.idealized.Field` is a collection of {class}`~tams.idealized.Blob`s on a grid that, together, combine to make a field that we can run {func}`tams.identify` on.

In [ ]:
Field()

### Single well

In [ ]:
%%time

b = Blob(center=(0, 0), width=10, height=6)

ctt = Field(b).to_xarray()

(ce,) = tams.identify(ctt)

fig, ax = plt.subplots(figsize=(10, 6))

ctt.plot(ax=ax)
gpd.GeoSeries(b.ring).plot(ec="magenta", ax=ax)

ce.plot(ax=ax, lw=2.5, ls=":", ec="black", fc="none", zorder=10)

ax.set_aspect("equal")

In [ ]:
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True, layout="constrained")

ctt.sel(lat=0, method="nearest").plot(ax=ax1)
ctt.sel(lon=0, method="nearest").plot(ax=ax2)

ax2.set_ylabel("")
for ax in fig.get_axes():
    ax.axhline(235, c="0.35", ls=":")

### Reverse well

In [ ]:
# Flipping depth and making CTT threshold greater than the background,
# we can get a reverse well.
ctt = Field(
    Blob(center=(0, 0), width=10, height=6, depth=-20)
).to_xarray(ctt_threshold=300)

# TAMS will still identify if we adjust the thresholds accordingly
(ce,) = tams.identify(ctt, ctt_threshold=300, ctt_core_threshold=310)

fig, ax = plt.subplots()

ctt.plot(ax=ax)
ce.plot(ax=ax, lw=2.5, ls=":", ec="black", fc="none");

### Additive

In [ ]:
fld = Field(
    [
        Blob(center=(0, 0), width=10, height=6),
        Blob(center=(5, 0), width=10, height=6),
    ]
)

fig, [ax1, ax2] = plt.subplots(2, 1, figsize=(8, 6), sharex=True, sharey=True, layout="constrained")

kws = dict(vmin=170, vmax=270)
for ax, ctt in zip([ax1, ax2], [fld.to_xarray(), fld.to_xarray(additive=True)]):
    ctt.plot(ax=ax, **kws)
    ax.text(0.01, 0.98, f"min: {ctt.min().item():.2f}", ha="left", va="top", transform=ax.transAxes)
    (ce,) = tams.identify(ctt)
    ce.plot(ax=ax, lw=2.5, ls=":", ec="black", fc="none")

ax1.set_xlabel("")

for ax in [ax1, ax2]:
    ax.set_aspect("equal")

## Sim

A {class}`~tams.idealized.Sim` is the evolution of a {class}`~tams.idealized.Field` in time.

In [ ]:
# Nothing (no blobs)
Sim().advance(2).to_xarray().plot(col="time");

In [ ]:
%%time

ctt = Sim(
    Field(
        [
            Blob(width=10).set_tendency(center=(2, 0)),
            Blob(center=(-20, -10), width=6).set_tendency(center=(5, 3), width=1),
        ],
        lat=(-15, 15, 61),
    ),
).advance(9).to_xarray()

ces = tams.identify(ctt)

fg = ctt.plot(col="time", cmap="inferno_r", col_wrap=5, aspect=1.35)
for ce, ax in zip(ces, fg.axs.flat):
    if not ce.empty:
        ce.plot(ax=ax, lw=2, ec="w", fc="none")

👆 In TAMS v0.2, we lose track of the system(s) the last few time steps because the blobs are partially out of the domain and we drop open contours by default in the {func}`~tams.identify` stage.
In this example, we could fix this by increasing the domain size when creating the {class}`~tams.idealized.Field`.

In [ ]:
tams.plot_tracked(
    tams.track(ces, ctt.time),
    size=5,
    alpha=0.35,
    add_colorbar=True,
    cmap="turbo",
)